# Ice Puzzle

![title](snow_1.png)

In [1]:
#Install packages
import numpy as np
import random

In [2]:

class Environment:
    def __init__(self,totems,ice,treasure,start,goal,length=10,height=5,punishment=-5,reward=100):
        # Define environment size
        self.length=length 
        self.height=height
        self.Map=np.array(np.zeros((height,length)))

        # Define Items on the Map
        self.start=start #Starting Point
        self.goal=goal
        self.totems=totems
        self.ice=ice
        self.treasure=treasure
        self.punishment=punishment
        self.reward=reward

        # Define Variables for Training
        self.actions=["Up","Down","Left","Right"]
        self.Q=None
        self.R=None
        self.S=None
        self.episode_actions=[]

        # IMPORTANT: self.S_dict IS NOT THE SAME VARIABLE AS self.S
        # self.S_dict is a dictionary that takes in a coordinate and transforms it to an index corresponding to the element in self.S
        self.S_dict={}

        # Define the next state because this map cannot utilise simple directions
        self.transitions=None

    def CreateMap(self): #Creating Map for visualization purposes only

        for i in self.totems:
            self.Map[i] = np.nan

        for i in self.treasure:
            self.Map[i] = self.reward

        for i in self.ice:
            self.Map[i] = -1

        return self.Map

    #Creating State Matrices and its corresponding index in self.S_dict
    def CreateStateMat(self): 
        self.S=[]
        self.S_dict={}
        iter=0
        for i in range(self.height):
            for j in range(self.length):
                if((i,j) not in self.totems+self.ice):
                    # self.S_dict.update({iter:(i,j)})
                    self.S.append((i,j))
                    self.S_dict.update({'{},{}'.format(i,j):iter})
                    iter+=1

        return self.S_dict

    #Defining the Transition Matrix
    def CreateTransitions(self): 
        self.transitions=np.full((len(self.R),len(self.actions)),np.nan)
        return self.transitions

    def GiveTransitions(self,loc,acts,destinations):
        # Variables:
        # 1) loc: Coordinate in the form of 'x,y' INCLUDING the apostrophes, 
        # which converts to its corresponding index through self.s_dict
        # 2) acts: Possible actions in the form of ['Up','Down,'Left','Right'] where any of the directions 
        # can be removed to indicate the action is not possible
        # 3) destinations: End coordinate after taking the action defined above in the form of ['x,y']
        # Shapes of acts and destinations must be the same

        if(len(acts)!=len(destinations)):
            raise Exception('Action List must be equal length to Destination List')

        for iter,x in enumerate(acts):
            self.transitions[self.S_dict[loc[0]],self.actions.index(x)]=self.S_dict[destinations[iter]]
        return

    def CreateRMat(self):
        self.R=np.full((len(self.S_dict),len(self.actions)),np.nan)
        return self.R

    def CreateQMat(self):
        self.Q=np.zeros((len(self.S_dict),len(self.actions)))

    def GiveActions(self,loc,acts):

        # loc and acts are the same variables defined in GiveTransitions()

        for iter,x in enumerate(acts):
                self.R[self.S_dict[loc[0]],self.actions.index(x)]=0
        return

    #Assign reward value of punishment value
    def GiveValue(self,loc,acts,value=100): 
        self.R[self.S_dict[loc[0]],self.actions.index(acts)]=value

    def TrainEpisode(self,alpha,gamma,epsilon,max_step,default_start=True): 
        #...
        # Training Episodes using Code from INM707 Lab 4 as reference
        #...
        if(default_start==True):
            curr_s=self.S_dict[self.start]
        else:
            curr_s=random.randint(0,len(self.S_dict)-1)
        # print("Starting state is '{}'".format(list(self.S_dict.keys())[curr_s]))
        self.episode_actions=[]
        for step in range(max_step):

            # Define actions for both exploring and exploiting policies
            open_actions = np.where(~np.isnan(self.R[curr_s]))[0]
            # print([self.actions[a] for a in open_actions])

            open_q = [self.Q[curr_s,a] for a in open_actions]

            best_act = open_actions[np.where(open_q == np.max(open_q))[0]]
            best_act_q = [self.Q[curr_s,x] for x in best_act]

            # print(best_act)

            # Pick Action based on policy
            if np.random.uniform() < epsilon:
                a = np.random.choice(open_actions)
            else:
                a = np.random.choice(best_act)

            # Update Environment States
            r = self.R[curr_s,a]
            s_old = curr_s
            curr_s = int(self.transitions[curr_s,a])

            self.episode_actions.append("{}, {}".format(self.S[s_old],self.actions[a]))
            # print("New state is '{}'".format(list(self.S_dict.keys())[int(curr_s)]))
            # print((self.Q[curr_s]))
            q_updated =  self.Q[s_old,a] + alpha*(self.R[s_old,a] + gamma*np.max(self.Q[curr_s]) - self.Q[s_old,a])
            self.Q[s_old,a] = q_updated

            # print('Q matrix updated: \n\n {}'.format(self.Q.round(0)))

            if curr_s == self.S_dict[self.goal]:
                # print("Goal state '{}' reached. Ending episode.".format(self.goal))
                break

        return self.episode_actions

# Create Map

In [3]:
#Define Totems (Tiles that cant be stood on), Ice (Tiles that can only be slid past), Treasure(Tile containing reward) and Start(Starting Point)
Totems=[(0,0),(1,0),(2,0),(0,2),(4,1),(4,5),(3,7),(0,9),(2,9),(3,9),(4,9)]
Ice=[(2,1),(2,2),(3,2),(1,3),(2,3),(3,3),(1,4),(2,4),(3,4),(0,5),(1,5),(2,5),(3,5),(2,7),(2,8),(1,6),(2,6),(0,7),(1,7),(4,7),(1,8),(3,8)]
Treasure=[(1,9)
Start='3,0'
End = '1,9']
Start='3,0'
End = '1,9'

IceEnv = Environment(Totems,Ice,Treasure,Start,End)
Map = IceEnv.CreateMap()
print(Map)

#Create State Matrix
S = IceEnv.CreateStateMat()

[[ nan   0.  nan   0.   0.  -1.   0.  -1.   0.  nan]
 [ nan   0.   0.  -1.  -1.  -1.  -1.  -1.  -1. 100.]
 [ nan  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  nan]
 [  0.   0.  -1.  -1.  -1.  -1.   0.  nan  -1.  nan]
 [  0.  nan   0.   0.   0.  nan   0.  -1.   0.  nan]]


In [4]:
# Manually Define R matrix
R=IceEnv.CreateRMat()

IceEnv.GiveActions(["0,1"],["Down"]) #(0,1)
IceEnv.GiveActions(["0,3"],["Down","Right"]) #(0,3) 
IceEnv.GiveActions(["0,4"],["Down","Left","Right"]) #(0,4)
IceEnv.GiveActions(["0,6"],["Down","Left","Right"]) #(0,6)
IceEnv.GiveActions(["0,8"],["Down","Left"]) #(0,8)
IceEnv.GiveActions(["1,1"],["Up","Down","Right"]) #(1,1)
IceEnv.GiveValue(["1,1"],"Right",100) #Rewards for Going Right from (1,1)
IceEnv.GiveActions(["1,2"],["Down","Left","Right"]) #(1,2)
IceEnv.GiveValue(["1,2"],"Right",100) #Rewards for Going Right from (1,1)
IceEnv.GiveActions(["1,9"],["Up","Down","Left","Right"]) #(1,9)
IceEnv.GiveActions(["3,0"],["Down","Right"]) #(3,0)
IceEnv.GiveActions(["3,1"],["Up","Left","Right"]) #(3,1)
IceEnv.GiveActions(["3,6"],["Up","Down","Left"]) #(3,6)
IceEnv.GiveActions(["4,0"],["Up"]) #(4,0)
IceEnv.GiveActions(["4,2"],["Up","Right"]) #(4,2)
IceEnv.GiveActions(["4,3"],["Up","Left","Right"]) #(4,3)
IceEnv.GiveActions(["4,4"],["Up","Left"]) #(4,4)
IceEnv.GiveActions(["4,6"],["Up","Right"]) #(4,6)
IceEnv.GiveActions(["4,8"],["Up","Left"]) #(4,8)


print(IceEnv.R)

[[ nan   0.  nan  nan]
 [ nan   0.  nan   0.]
 [ nan   0.   0.   0.]
 [ nan   0.   0.   0.]
 [ nan   0.   0.  nan]
 [  0.   0.  nan 100.]
 [ nan   0.   0. 100.]
 [  0.   0.   0.   0.]
 [ nan   0.  nan   0.]
 [  0.  nan   0.   0.]
 [  0.   0.   0.  nan]
 [  0.  nan  nan  nan]
 [  0.  nan  nan   0.]
 [  0.  nan   0.   0.]
 [  0.  nan   0.  nan]
 [  0.  nan  nan   0.]
 [  0.  nan   0.  nan]]


In [5]:
# Manually Defining the state transititons from tile to tile
IceEnv.CreateTransitions()
IceEnv.GiveTransitions(["0,1"],["Down"],["0,3"]) #(0,1)
IceEnv.GiveTransitions(["0,3"],["Down","Right"],["4,3","0,8"]) #(0,3) 
IceEnv.GiveTransitions(["0,4"],["Down","Left","Right"],["4,4","0,3","0,8"]) #(0,4)
IceEnv.GiveTransitions(["0,6"],["Down","Left","Right"],["4,6","0,3","0,8"]) #(0,6)
IceEnv.GiveTransitions(["0,8"],["Down","Left"],["4,8","0,3"]) #(0,8)
IceEnv.GiveTransitions(["1,1"],["Up","Down","Right"],["0,1","3,1","1,9"]) #(1,1)
IceEnv.GiveTransitions(["1,2"],["Down","Left","Right"],["4,2","1,1","1,9"]) #(1,2)
IceEnv.GiveTransitions(["1,9"],["Up","Down","Left","Right"],["1,9","1,9","1,9","1,9"]) #(1,9)
IceEnv.GiveTransitions(["3,0"],["Down","Right"],["4,0","3,6"]) #(3,0)
IceEnv.GiveTransitions(["3,1"],["Up","Left","Right"],["0,1","3,0","3,6"]) #(3,1)
IceEnv.GiveTransitions(["3,6"],["Up","Down","Left"],["3,0","4,6","0,6"]) #(3,6)
IceEnv.GiveTransitions(["4,0"],["Up"],["3,0"]) #(4,0)
IceEnv.GiveTransitions(["4,2"],["Up","Right"],["1,2","4,4"]) #(4,2)
IceEnv.GiveTransitions(["4,3"],["Up","Left","Right"],["0,3","4,2","4,4"]) #(4,3)
IceEnv.GiveTransitions(["4,4"],["Up","Left"],["0,4","4,2"]) #(4,4)
IceEnv.GiveTransitions(["4,6"],["Up","Right"],["0,6","4,8"]) #(4,6)
IceEnv.GiveTransitions(["4,8"],["Up","Left"],["0,8","4,6"]) #(4,8)
print(IceEnv.transitions)

[[nan  1. nan nan]
 [nan 13. nan  4.]
 [nan 14.  1.  4.]
 [nan 15.  1.  4.]
 [nan 16.  1. nan]
 [ 0.  9. nan  7.]
 [nan 12.  5.  7.]
 [ 7.  7.  7.  7.]
 [nan 11. nan 10.]
 [ 0. nan  8. 10.]
 [ 8. 15.  3. nan]
 [ 8. nan nan nan]
 [ 6. nan nan 14.]
 [ 1. nan 12. 14.]
 [ 2. nan 12. nan]
 [ 3. nan nan 16.]
 [ 4. nan 15. nan]]


# Read map from file

In [17]:
lines = open('snow_map').readlines()

# Remove first line
lines = lines[1:]

map = []
for line in lines:
    map.append(line[1:-1])

 0123456789
0x x      x
1x        r
2         x
3       x  
4     x   x


In [ ]:
N = len(map)
M = len(map[0])

dirs = {
    'Up':    (-1,  0),
    'Down':  ( 1,  0),
    'Left':  ( 0, -1),
    'Right': ( 0,  1),
}
for i in range(N):
    for j in range(M):
        if map[i][j] == 'x':
            continue

        for dir, (dy, dx) in dirs.items():
            while y >= 0 and y < N and x >= 0 and x < M and map[y][x] != 'x':
                y = i + dy
                x = j + dx

            if y != i and x != j:
                y -= dy
                x -= dx

                IceEnv.GiveTransitions([f'{i},{j}'], [dir], [f'{y},{x}'])

# Begin Reinforcement Learning

## Initialize State

In [6]:
import random

# Initialize Training
alpha = 1
gamma = 0.7
epsilon = 0.8
max_step = 500
decay=0.95

IceEnv.CreateQMat()

In [7]:
for i in range(100):
    actionlist=IceEnv.TrainEpisode(alpha,gamma,epsilon,max_step)
    epsilon*=decay

print(IceEnv.Q)
print(epsilon)
print(IceEnv.S_dict)
print('{} \n'.format([a for a in actionlist]))
# print('{} \n'.format([a for a in actionlist]))

[[  0.        24.01       0.         0.      ]
 [  0.        34.3        0.        16.807   ]
 [  0.         0.         0.        16.807   ]
 [  0.        11.7649    24.01      16.807   ]
 [  0.        11.7649    24.01       0.      ]
 [  0.         5.764801   0.       100.      ]
 [  0.        49.        70.       100.      ]
 [  0.         0.         0.         0.      ]
 [  0.         5.764801   0.        11.7649  ]
 [  0.         0.         8.23543    0.      ]
 [  8.23543   11.7649    16.807      0.      ]
 [  8.23543    0.         0.         0.      ]
 [ 70.         0.         0.        34.3     ]
 [ 24.01       0.        49.        34.3     ]
 [ 11.7649     0.        49.         0.      ]
 [ 16.807      0.         0.        11.7649  ]
 [ 16.807      0.        11.7649     0.      ]]
0.004736423376267195
{'0,1': 0, '0,3': 1, '0,4': 2, '0,6': 3, '0,8': 4, '1,1': 5, '1,2': 6, '1,9': 7, '3,0': 8, '3,1': 9, '3,6': 10, '4,0': 11, '4,2': 12, '4,3': 13, '4,4': 14, '4,6': 15, '4,8': 16}
[